Explicación de las Mejoras

Aumento del Número de Capas Convolucionales: Se añadió una tercera capa convolucional (conv3) con 128 filtros, lo que permite al modelo aprender características más complejas de las imágenes.

Aumento del Número de Filtros: Se incrementaron los filtros en cada capa convolucional para mejorar la capacidad de captura de características.

Dropout: Se añadieron capas de Dropout con una tasa del 50% (self.dropout) después de las capas completamente conectadas para prevenir el sobreajuste y mejorar la generalización del modelo.

Aumento de Datos: Se aplicaron técnicas de aumento de datos (flip horizontal aleatorio y rotación aleatoria) para generar más variaciones de las imágenes de entrenamiento, mejorando así la robustez del modelo.

Normalización: Se añadió una normalización para que los valores de los píxeles tengan una media de 0.1307 y una desviación estándar de 0.3081, lo que puede ayudar a mejorar la convergencia del entrenamiento.

Estos cambios deberían mejorar la capacidad del modelo para generalizar y proporcionar una mayor precisión en el conjunto de prueba.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Aumentación de datos
    transforms.RandomRotation(10),      # Aumentación de datos
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalización
])

# Cargamos el dataset MNIST
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Creamos los dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

# Definimos el modelo de red neuronal convolucional mejorado
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Capa adicional
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)  # Ajuste del tamaño
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

# Instanciamos la red neuronal
model = ImprovedCNN()

# Definimos la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Entrenamos el modelo
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Asegurarse de que el modelo está en modo entrenamiento
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluamos el modelo
model.eval()  # Asegurarse de que el modelo está en modo evaluación
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4885916.68it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 249751.44it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2051379.44it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4420076.28it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch [1/10], Step [100/938], Loss: 1.1176
Epoch [1/10], Step [200/938], Loss: 0.4457
Epoch [1/10], Step [300/938], Loss: 0.3560
Epoch [1/10], Step [400/938], Loss: 0.1786
Epoch [1/10], Step [500/938], Loss: 0.3843
Epoch [1/10], Step [600/938], Loss: 0.3106
Epoch [1/10], Step [700/938], Loss: 0.2547
Epoch [1/10], Step [800/938], Loss: 0.1104
Epoch [1/10], Step [900/938], Loss: 0.3607
Epoch [2/10], Step [100/938], Loss: 0.3363
Epoch [2/10], Step [200/938], Loss: 0.1463
Epoch [2/10], Step [300/938], Loss: 0.1420
Epoch [2/10], Step [400/938], Loss: 0.1870
Epoch [2/10], Step [500/938], Loss: 0.2105
Epoch [2/10], Step [600/938], Loss: 0.1569
Epoch [2/10], Step [700/938], Loss: 0.0423
Epoch [2/10], Step [800/938], Loss: 0.1901
Epoch [2/10], Step [900/938], Loss: 0.2015
Epoch [3/10], Step [100/938], Loss: 0.0408
Epoch [3/10], Step [200/938], Loss: 0.1878
Epoch [3/10], Step [300/938], Loss: 0.2574
Epoch [3/10], Step [40